# Regression Week 5: Feature Selection and LASSO (Interpretation)

In this notebook, you will use LASSO to select features, building on a pre-implemented solver for LASSO (using GraphLab Create, though you can use other solvers). You will:
* Run LASSO with different L1 penalties.
* Choose best L1 penalty using a validation set.
* Choose best L1 penalty using a validation set, with additional constraint on the size of subset.

In the second notebook, you will implement your own LASSO solver, using coordinate descent. 

# Fire up Graphlab Create

In [21]:
import sklearn, sframe, numpy as np

# Load in house sales data

Dataset is from house sales in King County, the region where the city of Seattle, WA is located.

In [2]:
import pandas as pd

dtype_dict = {'bathrooms':float, 'waterfront':int, 'sqft_above':int, 'sqft_living15':float, 'grade':int, 'yr_renovated':int, 'price':float, 'bedrooms':float, 'zipcode':str, 'long':float, 'sqft_lot15':float, 'sqft_living':float, 'floors':float, 'condition':int, 'lat':float, 'date':str, 'sqft_basement':int, 'yr_built':int, 'id':str, 'sqft_lot':int, 'view':int}

sales = pd.read_csv('kc_house_data.csv', dtype=dtype_dict)

# Create new features

As in Week 2, we consider features that are some transformations of inputs.

In [3]:
from math import log, sqrt
sales['sqft_living_sqrt'] = sales['sqft_living'].apply(sqrt)
sales['sqft_lot_sqrt'] = sales['sqft_lot'].apply(sqrt)
sales['bedrooms_square'] = sales['bedrooms']*sales['bedrooms']

# In the dataset, 'floors' was defined with type string, 
# so we'll convert them to float, before creating a new feature.
sales['floors'] = sales['floors'].astype(float) 
sales['floors_square'] = sales['floors']*sales['floors']

* Squaring bedrooms will increase the separation between not many bedrooms (e.g. 1) and lots of bedrooms (e.g. 4) since 1^2 = 1 but 4^2 = 16. Consequently this variable will mostly affect houses with many bedrooms.
* On the other hand, taking square root of sqft_living will decrease the separation between big house and small house. The owner may not be exactly twice as happy for getting a house that is twice as big.

# Learn regression weights with L1 penalty

Let us fit a model with all the features available, plus the features we just created above.

In [4]:
all_features = ['bedrooms', 'bedrooms_square',
            'bathrooms',
            'sqft_living', 'sqft_living_sqrt',
            'sqft_lot', 'sqft_lot_sqrt',
            'floors', 'floors_square',
            'waterfront', 'view', 'condition', 'grade',
            'sqft_above',
            'sqft_basement',
            'yr_built', 'yr_renovated']

Applying L1 penalty requires adding an extra parameter (`l1_penalty`) to the linear regression call in GraphLab Create. (Other tools may have separate implementations of LASSO.)  Note that it's important to set `l2_penalty=0` to ensure we don't introduce an additional L2 penalty.

In [5]:
model_all = sklearn.linear_model.Lasso(alpha=5e2, normalize=True)
model_all.fit(sales[all_features], sales['price'])

Lasso(alpha=500.0, copy_X=True, fit_intercept=True, max_iter=1000,
   normalize=True, positive=False, precompute=False, random_state=None,
   selection='cyclic', tol=0.0001, warm_start=False)

Find what features had non-zero weight.

In [10]:
coef_dict = {}
for coef, feat in zip(model_all.coef_,all_features):
    coef_dict[feat] = coef
    print feat, coef

bedrooms 0.0
bedrooms_square 0.0
bathrooms 0.0
sqft_living 134.439313955
sqft_living_sqrt 0.0
sqft_lot 0.0
sqft_lot_sqrt 0.0
floors 0.0
floors_square 0.0
waterfront 0.0
view 24750.0045856
condition 0.0
grade 61749.1030907
sqft_above 0.0
sqft_basement 0.0
yr_built -0.0
yr_renovated 0.0


Note that a majority of the weights have been set to zero. So by setting an L1 penalty that's large enough, we are performing a subset selection. 

***QUIZ QUESTION***:
According to this list of weights, which of the features have been chosen? 

# Selecting an L1 penalty

To find a good L1 penalty, we will explore multiple values using a validation set. Let us do three way split into train, validation, and test sets:
* Split our sales data into 2 sets: training and test
* Further split our training data into two sets: train, validation

Be *very* careful that you use seed = 1 to ensure you get the same answer!

In [129]:
X_trainandvalid, X_test, y_trainandvalid, y_test = sklearn.model_selection.train_test_split(sales[all_features], sales['price'], test_size=0.1, random_state=1)
X_train, X_valid, y_train, y_valid = sklearn.model_selection.train_test_split(X_trainandvalid,y_trainandvalid, test_size=0.5, random_state=1)

Next, we write a loop that does the following:
* For `l1_penalty` in [10^1, 10^1.5, 10^2, 10^2.5, ..., 10^7] (to get this in Python, type `np.logspace(1, 7, num=13)`.)
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list.
    * Compute the RSS on VALIDATION data (here you will want to use `.predict()`) for that `l1_penalty`
* Report which `l1_penalty` produced the lowest RSS on validation data.

When you call `linear_regression.create()` make sure you set `validation_set = None`.

Note: you can turn off the print out of `linear_regression.create()` with `verbose = False`

In [131]:
for i in np.logspace(1, 7, num=13):
    model = sklearn.linear_model.Lasso(alpha=i, normalize=True)
    model.fit(X_train, y_train)
    predictions = model.predict(X_valid)
    RSS = ((predictions - y_valid) ** 2).sum() + i * np.sum(np.abs(model.coef_))
    print np.log10(i), RSS

1.0 4.16119319732e+14
1.5 4.2633613475e+14
2.0 4.59629632522e+14
2.5 5.09831299526e+14
3.0 7.11390968646e+14
3.5 1.2907582474e+15
4.0 1.2907582474e+15
4.5 1.2907582474e+15
5.0 1.2907582474e+15
5.5 1.2907582474e+15
6.0 1.2907582474e+15
6.5 1.2907582474e+15
7.0 1.2907582474e+15


In [132]:
model = sklearn.linear_model.Lasso(alpha=10, normalize=True)
model.fit(X_trainandvalid, y_trainandvalid)
predictions = model.predict(X_test)
RSS = ((predictions - y_test) ** 2).sum() + i * np.sum(np.abs(model.coef_))
print RSS

1.20680929992e+14


*** QUIZ QUESTION. *** What was the best value for the `l1_penalty`?

In [133]:
print np.count_nonzero(model.coef_) + np.count_nonzero(model.intercept_)
print model.coef_

16
[ -1.85011239e+04   1.68869996e+02   4.67439693e+04   4.69018686e+02
  -3.05901881e+04   3.14142507e-01  -4.41402511e+02  -0.00000000e+00
   5.88043360e+03   5.24698851e+05   4.32167477e+04   2.38322897e+04
   1.27013716e+05   0.00000000e+00   9.13095310e-01  -3.19744313e+03
   1.45187500e+01]


***QUIZ QUESTION***
Also, using this value of L1 penalty, how many nonzero weights do you have?

# Limit the number of nonzero weights

What if we absolutely wanted to limit ourselves to, say, 7 features? This may be important if we want to derive "a rule of thumb" --- an interpretable model that has only a few features in them.

In this section, you are going to implement a simple, two phase procedure to achive this goal:
1. Explore a large range of `l1_penalty` values to find a narrow region of `l1_penalty` values where models are likely to have the desired number of non-zero weights.
2. Further explore the narrow region you found to find a good value for `l1_penalty` that achieves the desired sparsity.  Here, we will again use a validation set to choose the best value for `l1_penalty`.

In [134]:
max_nonzeros = 7

## Exploring the larger range of values to find a narrow range with the desired sparsity

Let's define a wide range of possible `l1_penalty_values`:

In [135]:
l1_penalty_values = np.logspace(1, 4, num=13)

Now, implement a loop that search through this space of possible `l1_penalty` values:

* For `l1_penalty` in `np.logspace(8, 10, num=20)`:
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list. When you call `linear_regression.create()` make sure you set `validation_set = None`
    * Extract the weights of the model and count the number of nonzeros. Save the number of nonzeros to a list.
        * *Hint: `model['coefficients']['value']` gives you an SArray with the parameters you learned.  If you call the method `.nnz()` on it, you will find the number of non-zero parameters!* 

In [136]:
nonzero_list = np.zeros(len(l1_penalty_values))
for i in range(len(l1_penalty_values)):
    model = sklearn.linear_model.Lasso(alpha=l1_penalty_values[i], normalize=True)
    model.fit(X_train, y_train)
    nonzero = np.count_nonzero(model.coef_) + np.count_nonzero(model.intercept_)
    nonzero_list[i] = nonzero
print nonzero_list

[ 16.  16.  15.  11.  11.   7.   6.   6.   3.   3.   1.   1.   1.]


Out of this large range, we want to find the two ends of our desired narrow range of `l1_penalty`.  At one end, we will have `l1_penalty` values that have too few non-zeros, and at the other end, we will have an `l1_penalty` that has too many non-zeros.  

More formally, find:
* The largest `l1_penalty` that has more non-zeros than `max_nonzeros` (if we pick a penalty smaller than this value, we will definitely have too many non-zero weights)
    * Store this value in the variable `l1_penalty_min` (we will use it later)
* The smallest `l1_penalty` that has fewer non-zeros than `max_nonzeros` (if we pick a penalty larger than this value, we will definitely have too few non-zero weights)
    * Store this value in the variable `l1_penalty_max` (we will use it later)


*Hint: there are many ways to do this, e.g.:*
* Programmatically within the loop above
* Creating a list with the number of non-zeros for each value of `l1_penalty` and inspecting it to find the appropriate boundaries.

In [137]:
for i in range(len(nonzero_list)-1):
    if (nonzero_list[i] > max_nonzeros) and (nonzero_list[i+1] <= max_nonzeros):
        l1_penalty_min = l1_penalty_values[i]
        l1_penalty_max = l1_penalty_values[i+1]
print l1_penalty_min, l1_penalty_max

100.0 177.827941004


***QUIZ QUESTION.*** What values did you find for `l1_penalty_min` and `l1_penalty_max`, respectively? 

## Exploring the narrow range of values to find the solution with the right number of non-zeros that has lowest RSS on the validation set 

We will now explore the narrow region of `l1_penalty` values we found:

In [138]:
l1_penalty_values = np.linspace(l1_penalty_min,l1_penalty_max,20)

* For `l1_penalty` in `np.linspace(l1_penalty_min,l1_penalty_max,20)`:
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list. When you call `linear_regression.create()` make sure you set `validation_set = None`
    * Measure the RSS of the learned model on the VALIDATION set

Find the model that the lowest RSS on the VALIDATION set and has sparsity *equal* to `max_nonzeros`.

In [140]:
nonzero_list = np.zeros(len(l1_penalty_values))
for i in range(len(l1_penalty_values)):
    model = sklearn.linear_model.Lasso(alpha=l1_penalty_values[i], normalize=True)
    model.fit(X_train, y_train)
    nonzero = np.count_nonzero(model.coef_) + np.count_nonzero(model.intercept_)
    nonzero_list[i] = nonzero
    predictions = model.predict(X_valid)
    RSS = ((predictions - y_valid) ** 2).sum() + l1_penalty_values[i] * np.sum(np.abs(model.coef_))
    print nonzero, RSS, l1_penalty_values[i] * np.sum(np.abs(model.coef_))

11 4.59629632522e+14 60714796.9556
11 4.60668234801e+14 62622916.8385
11 4.61742584502e+14 64485596.5354
11 4.62852750589e+14 66302726.7628
11 4.63998759567e+14 68074291.046
11 4.65180605935e+14 69800317.9189
11 4.66398257799e+14 71480827.6173
11 4.67649611389e+14 73113014.6066
10 4.68518566761e+14 74792809.0988
9 4.6939643483e+14 76450104.6082
8 4.70224613018e+14 78109557.2151
8 4.71026779254e+14 79714849.3274
8 4.71849987357e+14 81282826.2033
8 4.72694325852e+14 82813234.2508
8 4.735597438e+14 84306193.4666
7 4.74357305206e+14 85752721.467
7 4.75082781639e+14 87153262.9058
7 4.75824688688e+14 88515085.7231
7 4.76582840464e+14 89838516.841
7 4.77357210971e+14 91123600.815


***QUIZ QUESTIONS***
1. What value of `l1_penalty` in our narrow range has the lowest RSS on the VALIDATION set and has sparsity *equal* to `max_nonzeros`?
2. What features in this model have non-zero coefficients?